In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math
import random
%load_ext autoreload
%autoreload 2

In [2]:
# Our libraries
from proj1_helpers import *
from proj1_input_man import *
from proj1_linear_model import *
from proj1_ridge_regress import *
from proj1_logistic import *

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
DATA_TRAIN_PATH = '../data/train.csv' # train data path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
print(tX.shape)

(250000, 30)


### Changing the labels to {0,1}

In [5]:
# change the array from -1 1 to 0 1
# label simplification
# y == 0 non detected Boson, y == 1 detected Boson
y_ = np.array([0 if l == -1 else 1 for l in y])

# All of the following parts are the same for the test set, should be functions

# less code = better code (in this case)

### Dividing the features by the number of jets

In [6]:
tX_0, tX_1, tX_2_3 = split_to_Jet_Num(tX)

In [7]:
tX_0, tX_1, tX_2, tX_3 = alternative_split_to_Jet_Num(tX)

In [8]:
for i in range(tX_1.shape[1]):
    print(np.std(tX_3[:,i]))

288.1303027432567
38.07417940661156
48.78977837574543
83.81996788992959
1.4698926707450346
317.2726868423743
2.8672697710964288
0.8272411967910458
32.29848326337556
150.13499581029197
1.2471331404825152
0.9433373230483033
0.3876270487619129
32.43010841851103
1.2047095881208543
1.8241134002031094
32.96582686964663
1.2176561938554824
1.8180600465624461
49.697288657467055
1.8123415757149903
158.8254499542401
76.3120342898988
1.5732280712204414
1.8067332213365404
40.12542350677477
1.775313755049286
1.8147406038684843
128.45133643867


### Dividing also the output by the type of particle

In [9]:
y_0,y_1,y_2_3 = split_labels_to_Jet_Num(y_,tX)

In [10]:
y_0, y_1, y_2, y_3 = alternative_split_labels_to_Jet_Num(y_, tX)

### Adding a column of zeros and ones to detect whether the mass has been measured or not

In [11]:
# take the indices where the mass is not calculated, add the column which has 0 in those indices
# and 1 everywhere else for all matrices 0,1,2_3
tX_0 = find_mass(tX_0)
tX_1 = find_mass(tX_1)
# tX_2_3 = find_mass(tX_2_3)
tX_2 = find_mass(tX_2)
tX_3 = find_mass(tX_3)

### Throwing away the outliers from the training data

In [12]:
tX_0, col_to_delete_0 = fix_array(tX_0, 0)
print(tX_0.shape)
tX_1, col_to_delete_1 = fix_array(tX_1, 1)
print(tX_1.shape)
tX_2, _ = fix_array(tX_2, 2)
tX_3, _ = fix_array(tX_3, 3)
print(tX_2)
print(tX_3)
# print(tX_2_3.shape)

[5, 6, 7, 13, 23, 24, 25, 26, 27, 28, 29]
(99913, 19)
[5, 6, 7, 13, 26, 27, 28]
(77544, 23)
[]
[]
[[ 1.00000e+00  1.38470e+02  5.16550e+01 ...  1.24000e+00 -2.47500e+00
   1.13497e+02]
 [ 1.00000e+00  1.48754e+02  2.88620e+01 ...  1.31000e-01 -2.76700e+00
   1.79877e+02]
 [ 1.00000e+00  1.41481e+02  7.36000e-01 ... -7.98000e-01 -2.78500e+00
   2.78009e+02]
 ...
 [ 1.00000e+00  1.19934e+02  2.00780e+01 ... -1.72500e+00 -2.75600e+00
   1.12938e+02]
 [ 1.00000e+00  1.26151e+02  2.90230e+01 ... -5.99000e-01 -2.52500e+00
   1.93099e+02]
 [ 1.00000e+00  1.15254e+02  4.71560e+01 ... -5.80000e-02 -1.13700e+00
   1.74176e+02]]
[[ 1.00000e+00  8.97440e+01  1.35500e+01 ...  2.24000e-01  3.10600e+00
   1.93660e+02]
 [ 1.00000e+00  1.14744e+02  1.02860e+01 ...  1.77300e+00 -2.07900e+00
   1.65640e+02]
 [ 1.00000e+00  1.21681e+02  6.04100e+00 ... -1.25700e+00 -6.09000e-01
   2.53461e+02]
 ...
 [ 1.00000e+00 -9.99000e+02  8.38710e+01 ...  3.07000e+00  1.61200e+00
   2.71833e+02]
 [ 1.00000e+00 -9.990

### Now we substitute the -999 values with the median

In [13]:
tX_0, column_median_0 = fix_median(tX_0)
tX_1, column_median_1 = fix_median(tX_1)
# tX_2_3 = fix_median(tX_2_3)
tX_2, column_median_2 = fix_median(tX_2)
tX_3, column_median_3 = fix_median(tX_3)

### Now we standardize the data

In [14]:
# tX_2_3[:,1:], mean_2_3, std_2_3 = standardize(tX_2_3[:,1:])
tX_3[:,1:], mean_3, std_3 = standardize(tX_3[:,1:])
tX_2[:,1:], mean_2, std_2 = standardize(tX_2[:,1:]) #we standardize everything a part from the column added manually
tX_0[:,1:], mean_0, std_0 = standardize(tX_0[:,1:])
tX_1[:,1:], mean_1, std_1 = standardize(tX_1[:,1:])

### We insert the column for the bias term

In [15]:
tX_tilda_0 = np.insert(tX_0, 0, np.ones(tX_0.shape[0]), axis=1)
tX_tilda_1 = np.insert(tX_1, 0, np.ones(tX_1.shape[0]), axis=1)
# tX_tilda_2_3 = np.insert(tX_2_3, 0, np.ones(tX_2_3.shape[0]), axis=1)
tX_tilda_2 = np.insert(tX_2, 0, np.ones(tX_2.shape[0]), axis=1)
tX_tilda_3 = np.insert(tX_3, 0, np.ones(tX_3.shape[0]), axis=1)

## Do your thing crazy machine learning thing here :) ...

## The following cells of code aim at finetuning the hyperparameters of the code

### Calculate the optimal degree for gradient descent

In [ ]:
degree_opt_0_GD = finetune_GD(tX_tilda_0, y_0)
degree_opt_1_GD = finetune_GD(tX_tilda_1, y_1)
degree_opt_2_3_GD = finetune_GD(tX_tilda_2_3, y_2_3)

/home/john/Documents/MachineLearning/Machine-Learning-P1/project1/scripts/proj1_linear_model.py:19: RuntimeWarning: invalid value encountered in matmul
  gradient = -(1/N) * (tx.T) @ (e) # calculate the gradient
/home/john/Documents/MachineLearning/Machine-Learning-P1/project1/scripts/proj1_linear_model.py:38: RuntimeWarning: invalid value encountered in subtract
  w = w - gamma * gradient # conduct a step of gradient descent


In [ ]:
degree_opt_1_GD = finetune_GD(tX_tilda_1, y_1)

In [ ]:
# This cell takes a long time to execute, the hyperparameters found are:
# degree =2 for tX_tilda_0 with an accuracy of 0.82694771 on the validation set
# degree =3 for tX_tilda_1 with an accuracy of 0.7937687 on the validation set
# degree =3 for tX_tilda_2_3 with an accuracy of 0.81856907 on the validation set

In [ ]:
w_GD_0 = optimal_weights_GD(tX_tilda_0,y_0,degree_opt_0_GD,lambda_opt_0_GD)
w_GD_1 = optimal_weights_GD(tX_tilda_1,y_1,degree_opt_1_GD,lambda_opt_1_GD)
w_GD_2_3 = optimal_weights_GD(tX_tilda_2_3,y_2_3,degree_opt_2_3_GD,lambda_opt_2_3_GD)

### Calculate the optimal degree for stochastic gradient descent for feature augmentation

In [ ]:
degree_opt_0_SGD = finetune_SGD(tX_tilda_0, y_0)
degree_opt_1_SGD = finetune_SGD(tX_tilda_1, y_1)
degree_opt_2_SGD = finetune_SGD(tX_tilda_2, y_2)
degree_opt_3_SGD = finetune_SGD(tX_tilda_3, y_3)

### Calculate the optimal lambda for ridge regression and the optimal degree for feature augmentation

In [ ]:
lambda_opt_0_ridge, degree_opt_0_ridge = finetune_ridge(tX_tilda_0, y_0, degrees=np.array([2]), lambdas = np.logspace(-5, -1, 10), crossing=True)
lambda_opt_1_ridge, degree_opt_1_ridge = finetune_ridge(tX_tilda_1, y_1, degrees= np.array([2]), lambdas = np.logspace(-5, -1, 10), crossing=True)
lambda_opt_2_ridge, degree_opt_2_ridge = finetune_ridge(tX_tilda_2, y_2, degrees=np.array([2]), lambdas = np.logspace(-5, -1, 10), crossing=True)
lambda_opt_3_ridge, degree_opt_3_ridge = finetune_ridge(tX_tilda_3, y_3, degrees=np.array([2]), lambdas = np.logspace(-5, -1, 10), crossing=True)

In [ ]:
# to check
print(lambda_opt_0_ridge, degree_opt_0_ridge)
print(lambda_opt_1_ridge, degree_opt_1_ridge)
# print(lambda_opt_2_3_ridge, degree_opt_2_3_ridge)
print(lambda_opt_2_ridge, degree_opt_2_ridge)
print(lambda_opt_3_ridge, degree_opt_3_ridge)

In [ ]:
#Summarizing the best parameters found are
# lambda = 0.00316228 and degree=6 for tX_tilda_0 with a validation accuracy of 0.83847463
# [0.00558459] [6] gives accuracy = 0.83814433 for tX_tilda_0
# lambda = 0.00316228 and degree= 6 for tX_tilda_1 with a validation accuracy of 0.80354656
# lambda = 0.0047263 and degree = 6 for tX_tilda_3 gives 0.83759025 approximately
# lambda = 0.00546996 and degree = 5 for tX_tilda_2 gives 0.82747395
# lambda=1e-5 and degree=6 for tX_tilda_2_3 with a validation accuracy of 0.83021781

### Get the optimal weights with the calculated hyper parameters 

In [ ]:
# w_ridge_0 = optimal_weights_ridge(tX_tilda_0, y_0, degree_opt_0_ridge, lambda_opt_0_ridge)
# w_ridge_1 = optimal_weights_ridge(tX_tilda_1, y_1, degree_opt_1_ridge, lambda_opt_1_ridge)
w_ridge_1 = optimal_weights_ridge(tX_tilda_1, y_1, 6, 0.00774608)
# w_ridge_2_3 = optimal_weights_ridge(tX_tilda_2_3, y_2_3, degree_opt_2_3_ridge, lambda_opt_2_3_ridge)
# w_ridge_2 = optimal_weights_ridge(tX_tilda_2, y_2, degree_opt_2_ridge, lambda_opt_2_ridge)
# w_ridge_2_3 = optimal_weights_ridge(tX_tilda_3, y_3, degree_opt_3_ridge, lambda_opt_3_ridge)

### Calculate the optimal lambda for logistic regression and the optimal degree for feature augmentation

In [ ]:
lambda_opt_0_logistic, degree_opt_0_logistic = finetune_logistic(tX_tilda_0, y_0, 1.7783e-04, np.array([2]) , np.logspace(-2, 1, 7), 4, True)
lambda_opt_1_logistic, degree_opt_1_logistic = finetune_logistic(tX_tilda_1, y_1, 3.16228e-03, np.array([2]), np.logspace(-2, 1, 7), 4, True)
lambda_opt_2_logistic, degree_opt_2_logistic = finetune_logistic(tX_tilda_2, y_2, 1.7783e-04, np.array([2]), np.logspace(-3, 1, 7), 4, True)
lambda_opt_3_logistic, degree_opt_3_logistic = finetune_logistic(tX_tilda_3, y_3, 1.7783e-04, np.array([2]), np.logspace(-3, 1, 7), 4, True)   

In [ ]:
### Optimal values for logistic regression with feature crossing expansion of order 2 expanding also the bias term and the mass 
### for tX_tilda_0 lambda = 0.03162278 and degree = 2 with accuracy 0.8195411962527024
### for tX_tilda_1 lambda = 2.11944285 and degree = 2 with accuracy 0.7975343031053337
### for tX_tilda_2 lambda = 0.02154435 and degree = 2 with accuracy 0.8323606479275846
### for tX_tilda_3 lambda = 0.46415888 and degree = 2 with accuracy 0.8326565601876917

### Optimal values for logistic regression with feature crossing expansion of order 2 not expanding bias and mass
### for tX_tilda_0 lambda = 3.16227766 and degree = 2 with accuracy 0.824855873163882
### for tX_tilda_1 lambda = 1 and degree = 2 with accuracy 0.797585886722377
### for tX_tilda_2 lambda = 10 and degree = 2 with accuracy 0.830757393997142
### for tX_tilda_3 lambda = 10 and degree = 2 with accuracy 0.831573723154665

### Optimal values for logistic regression with feature crossing expansion of order 2 not expanding only the bias
### for tX_tilda_0 lambda = 0.1 and degree = 2 with accuracy 0.8224937945391945
### for tX_tilda_1 lambda = 1 and degree = 2 with accuracy 0.7975600949138554
### for tX_tilda_2 lambda = 10 and degree = 2 with accuracy 0.8321025885342226
### for tX_tilda_3 lambda = 10 and degree = 2 with accuracy 0.8334686879624617


### Best logistic regression models for each one of the four matrices
### for tX_tilda_0 without crossing feature expansion, normal expansion of degree = 2 and lambda = 1.95096 accuracy = 0.827668
### for tX_tilda_1 with crossing feature expansion expanding the mass, degree = 2 and lambda = 1 accuracy = 0.797585886722377
### for tX_tilda_2 with crossing feature expansion expanding the mass, degree = 2 and lambda = 10 accuracy = 0.832102588
### for tX_tilda_3 with crossing feature expansion expanding the mass, degree = 2 and lambda = 10 accuracy = 0.83346868

### best results on the best run on Aicrowd
### lambda = 1 and degree = 2 for tX_tilda_0
### lambda = 1.5 and degree = 3 for tX_tilda_1
### lambda = 0 and degree = 2 for tX_tilda_2_3

In [ ]:
lambda_opt_1_logistic,degree_opt_1_logistic = finetune_logistic(tX_tilda_1, y_1, gamma = 1.7783e-04, degrees = np.arange(1, 4), lambdas = random_interval(1.0e-07, 1.0e-02, 4))

In [ ]:
#Summarizing the best parameters found are
# lambda=1.95096 and degree=2 for tX_tilda_0 with a validation accuracy of 0.827668 #lambda =9.64683289, acc=0.82664
# lambda=1.6672 and degree=2 for tX_tilda_1 with a validation accuracy of 0.79229 #lamda = 13.00571 acc = 0.792466 gamma = 0.0004
# lambda=0.6835 and degree=2 for tX_tilda_2_3 with a validation accuracy of 0.8151
# lambda = 10 and degree = 2 for tX_tilda_2 gives an accuracy 0.8153088 
# lambda = 0.001 and degree = 2 for tX_tilda_3 gives an accuracy 0.8231366 

In [ ]:
# we will store in a dictionary for each predictor the parameters of the predictor, the first element of the list will be the lambda and the second
# will be the degree.

params_0 = {'model_1':[2,9.58856], 'model_2':[2, 7.04376], 'model_3': [2, 3.76039]}
params_1 = {'model_1':[2, 2.2889], 'model_2':[2, 13.89056], 'model_3': [2, 3.54514]}
params_2 = {'model_1':[2, 7.33430], 'model_2':[2, 12.49542], 'model_3': [2, 5.17695]}
params_3 = {'model_1':[2, 3.91067], 'model_2':[2, 13.78283], 'model_3': [2, 18.61038]}

### Get the optimal weights with the calculated hyper parameters 

In [ ]:
#we now compute the weights for each model and we store in a dictionary
gamma = 0.00017783
print(params_0['model_1'][0],params_0['model_1'][1])
w_logistic_0_model_1 = optimal_weights_logistic(tX_tilda_0, y_0, gamma, params_0['model_1'][0], params_0['model_1'][1])
w_logistic_1_model_1 = optimal_weights_logistic(tX_tilda_1, y_1, gamma, params_1['model_1'][0], params_1['model_1'][1])
w_logistic_2_model_1 = optimal_weights_logistic(tX_tilda_2, y_2, gamma, params_2['model_1'][0], params_2['model_1'][1])
w_logistic_3_model_1 = optimal_weights_logistic(tX_tilda_3, y_3, gamma, params_3['model_1'][0], params_3['model_1'][1])

w_logistic_0_model_2 = optimal_weights_logistic(tX_tilda_0, y_0, gamma, params_0['model_2'][0], params_0['model_2'][1])
w_logistic_1_model_2 = optimal_weights_logistic(tX_tilda_1, y_1, gamma, params_1['model_2'][0], params_1['model_2'][1])
w_logistic_2_model_2 = optimal_weights_logistic(tX_tilda_2, y_2, gamma, params_2['model_2'][0], params_2['model_2'][1])
w_logistic_3_model_2 = optimal_weights_logistic(tX_tilda_3, y_3, gamma, params_3['model_2'][0], params_3['model_2'][1])

w_logistic_0_model_3 = optimal_weights_logistic(tX_tilda_0, y_0, gamma, params_0['model_3'][0], params_0['model_3'][1])
w_logistic_1_model_3 = optimal_weights_logistic(tX_tilda_1, y_1, gamma, params_1['model_3'][0], params_1['model_3'][1])
w_logistic_2_model_3 = optimal_weights_logistic(tX_tilda_2, y_2, gamma, params_2['model_3'][0], params_2['model_3'][1])
w_logistic_3_model_3 = optimal_weights_logistic(tX_tilda_3, y_3, gamma, params_3['model_3'][0], params_3['model_3'][1])

### calculate the optimal lambda for logistic regression with batch

In [ ]:
lambda_opt_0_logistic_batch,degree_opt_0_logistic_batch = finetune_batch_logistic(tX_tilda_0, y_0, gamma = 1.7783e-04, degrees =np.arange(1, 4) , lambdas=random_interval(0.8, 1.5, 5) )

# Generate predictions and save ouput in csv format for submission:

In [ ]:
# open the test file
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
print(tX_test.shape)

We will now format the tX_test as we did for tX_train

### we split the test into the three subgroups

In [ ]:
tX_test_0, tX_test_1, tX_test_2, tX_test_3 = alternative_split_to_Jet_Num(tX_test)

### Adding a column of zeros and ones to detect whether the mass has been measured or not
This should be done prior to splitting it is the same procedure and just wastes space 

In [ ]:
# take the indices where the mass is not calculated, add the column which has 0 in those indices
# and 1 everywhere else for all matrices 0,1,2_3
tX_test_0 = find_mass(tX_test_0)
tX_test_1 = find_mass(tX_test_1)
tX_test_2 = find_mass(tX_test_2)
tX_test_3 = find_mass(tX_test_3)

### We drop the same columns we have dropped for the X training

In [ ]:
# tX_test_0 = fix_array(tX_test_0, 0)
# print(tX_test_0.shape)
# tX_test_1 = fix_array(tX_test_1, 1)
# print(tX_test_1.shape)

In [ ]:
tX_test_0 = np.delete(tX_test_0, col_to_delete_0, axis=1)
tX_test_1 = np.delete(tX_test_1, col_to_delete_1, axis=1)

### Now we substitute the -999 values with the median
This should also be done with a function it is the same thing repeated thrice

In [ ]:
tX_test_0 = fix_median_test(tX_test_0, column_median_0)
tX_test_1 = fix_median_test(tX_test_1, column_median_1)
tX_test_2 = fix_median_test(tX_test_2, column_median_2)
tX_test_3 = fix_median_test(tX_test_3, column_median_3)

In [ ]:
print(tX_test_0.shape)
print(w_logistic_0.shape)

### We standardize the test set using the mean and the standard deviation of the training

In [ ]:
print(tX_test_0.shape)

In [ ]:
print(tX_0.shape)

In [ ]:
# standardize the data in the test set
# should have used the same function both here and on the training part same process this is reduntant
def standardize_test(x, mean, std):
    """Standardize the test set."""
    x = x - mean 
    x = x / std
    return x

In [ ]:
tX_test_0[:,1:] = standardize_test(tX_test_0[:,1:], mean_0, std_0)  #we standardize everything a part from the column added manually
tX_test_1[:,1:] = standardize_test(tX_test_1[:,1:], mean_1, std_1)  #we standardize everything a part from the column added manually
tX_test_2[:,1:] = standardize_test(tX_test_2[:,1:], mean_2, std_2) #we standardize everything a part from the column added manually
tX_test_3[:,1:] = standardize_test(tX_test_3[:,1:], mean_3, std_3)

### We insert the column for the bias term

In [ ]:
tX_tilda_test_0 = np.insert(tX_test_0, 0, np.ones(tX_test_0.shape[0]), axis=1) #the first column now is all ones and is used for bias
tX_tilda_test_1 = np.insert(tX_test_1, 0, np.ones(tX_test_1.shape[0]), axis=1) #the first column now is all ones and is used for bias
tX_tilda_test_2 = np.insert(tX_test_2, 0, np.ones(tX_test_2.shape[0]), axis=1) #the first column now is all ones and is used for bias
tX_tilda_test_3 = np.insert(tX_test_3, 0, np.ones(tX_test_3.shape[0]), axis=1)

### We make the predictions with GD

In [ ]:
predictions_GD_0 = predict_GD(tX_tilda_test_0,w_GD_0,degree_opt_0_GD)
predictions_GD_1 = predict_GD(tX_tilda_test_1,w_GD_1,degree_opt_1_GD)
predictions_GD_2_3 = predict_GD(tX_tilda_test_2_3,w_GD_2_3,degree_opt_2_3_GD)

### We make the predictions with ridge regression

In [ ]:
predictions_ridge_0 = predict_ridge(tX_tilda_test_0, w_ridge_0,degree_opt_0_ridge)
predictions_ridge_1 = predict_ridge(tX_tilda_test_1, w_ridge_1,degree_opt_1_ridge)
predictions_ridge_2 = predict_ridge(tX_tilda_test_2, w_ridge_2, degree_opt_2_ridge)
predictions_ridge_3 = predict_ridge(tX_tilda_test_3, w_ridge_3, degree_opt_3_ridge)

### Predictions with logistic regression

In [ ]:
predictions_logistic_0 = predict_logistic(tX_tilda_test_0, w_logistic_0, degree_opt_0_logistic, False)
predictions_logistic_1 = predict_logistic(tX_tilda_test_1, w_logistic_1, degree_opt_1_logistic, True)
predictions_logistic_2 = predict_logistic(tX_tilda_test_2, w_logistic_2, degree_opt_2_logistic, True)
predictions_logistic_3 = predict_logistic(tX_tilda_test_3, w_logistic_3, degree_opt_3_logistic, True)

In [ ]:
print(params_0['model_1'][1])
print(w_logistic_0_model_1.shape)
print(tX_tilda_test_0.shape)
predictions_logistic_0_model_1 = predict_logistic(tX_tilda_test_0, w_logistic_0_model_1, params_0['model_1'][1])
predictions_logistic_1_model_1 = predict_logistic(tX_tilda_test_1, w_logistic_1_model_1, params_1['model_1'][1])
predictions_logistic_2_model_1 = predict_logistic(tX_tilda_test_2, w_logistic_2_model_1, params_2['model_1'][1])
predictions_logistic_3_model_1 = predict_logistic(tX_tilda_test_3, w_logistic_3_model_1, params_3['model_1'][1])

predictions_logistic_0_model_2 = predict_logistic(tX_tilda_test_0, w_logistic_0_model_2, params_0['model_2'][1])
predictions_logistic_1_model_2 = predict_logistic(tX_tilda_test_1, w_logistic_1_model_2, params_1['model_2'][1])
predictions_logistic_2_model_2 = predict_logistic(tX_tilda_test_2, w_logistic_2_model_2, params_2['model_2'][1])
predictions_logistic_3_model_2 = predict_logistic(tX_tilda_test_3, w_logistic_3_model_2, params_3['model_2'][1])

predictions_logistic_0_model_3 = predict_logistic(tX_tilda_test_0, w_logistic_0_model_3, params_0['model_3'][1])
predictions_logistic_1_model_3 = predict_logistic(tX_tilda_test_1, w_logistic_1_model_3, params_1['model_3'][1])
predictions_logistic_2_model_3 = predict_logistic(tX_tilda_test_2, w_logistic_2_model_3, params_2['model_3'][1])
predictions_logistic_3_model_3 = predict_logistic(tX_tilda_test_3, w_logistic_3_model_3, params_3['model_3'][1])

average_predictions_0 = ensemble_logistic(predictions_logistic_0_model_1, predictions_logistic_0_model_2, predictions_logistic_0_model_3)
average_predictions_1 = ensemble_logistic(predictions_logistic_0_model_1, predictions_logistic_0_model_2, predictions_logistic_0_model_3)
average_predictions_2 = ensemble_logistic(predictions_logistic_0_model_1, predictions_logistic_0_model_2, predictions_logistic_0_model_3)
average_predictions_3 = ensemble_logistic(predictions_logistic_0_model_1, predictions_logistic_0_model_2, predictions_logistic_0_model_3)

Now we have to reconstruct a single vector of predictions

### Generate final prediction list and print it in the file

In [ ]:
# final_predictions_GD = create_output(tX_test,predictions_GD_0,predictions_GD_1,predictions_GD_2_3)

In [ ]:
# final_predictions_ridge = create_output(tX_test,predictions_ridge_0,predictions_ridge_1,predictions_ridge_2_3)

In [ ]:
# final_predictions_logistic = create_output(tX_test,predictions_logistic_0,predictions_logistic_1,predictions_logistic_2_3)

In [ ]:
final_mixed_predictions = create_output(tX_test, predictions_logistic_0, predictions_logistic_1, predictions_logistic_2, predictions_logistic_3)

In [ ]:
print(final_mixed_predictions)
print(1 / len(final_mixed_predictions) * np.count_nonzero(final_mixed_predictions == 1))

In [ ]:
OUTPUT_PATH_GD = '../data/submission_GD.csv' # name towards GD output 
OUTPUT_PATH_RIDGE = '../data/submission_ridge.csv' # name towards ridge output 
OUTPUT_PATH_LOGISTIC = '../data/submission_logistic.csv' # name towards logistic output 
#create_csv_submission(ids_test, final_predictions_GD, OUTPUT_PATH_GD) # print csv file according to results
# create_csv_submission(ids_test, final_predictions_ridge, OUTPUT_PATH_RIDGE) # print csv file according to results
#create_csv_submission(ids_test, final_predictions_logistic, OUTPUT_PATH_LOGISTIC)
create_csv_submission(ids_test, final_mixed_predictions, OUTPUT_PATH_LOGISTIC)